# One-Zone Model Degeneracies

Exploring degeneracies between the star formation history (SFH) and Type Ia supernova delay time distribution (DTD) in abundance tracks and metallicity distribution functions (MDFs).

## Preamble

## Defining Star Formation Histories

## Defining Delay Time Distributions

## Results

## Discussion